In [226]:
from sklearn.tree import DecisionTreeClassifier as Tree
from sklearn.ensemble import RandomForestClassifier as Forest
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import time
%matplotlib qt

In [227]:
def Rotate(coords,theta):
    out = np.zeros(coords.shape)
    out[:,0] = coords[:,0]*np.cos(theta) + coords[:,1]*np.sin(theta)
    out[:,1] = -coords[:,0]*np.sin(theta) + coords[:,1]*np.cos(theta)
    return out

In [228]:
train_data = pd.read_csv('train.csv')
train_labels = pd.read_csv('train_label.csv')
N = train_data.shape[0]

In [231]:
X = train_data['longitude']
Y = train_data['latitude']
XY = np.array([X,Y]).T
Ru = np.random.uniform(0.0,1.0,size=N) # random continuous data
Rb = np.random.binomial(1,0.5,size=N) # random binary data
not_GPS_data = np.array([Ru,Rb]).T
target = np.zeros(X.shape[0],dtype=np.int)
target[np.array(train_labels['status_group']=='non functional')] = 0
target[np.array(train_labels['status_group']=='functional needs repair')] = 1
target[np.array(train_labels['status_group']=='functional')] = 2

In [416]:
s = 0.8
random_indices = np.arange(0,N)
np.random.shuffle(random_indices)
train_ind = random_indices[0:int(s*N)]
test_ind = random_indices[int(s*N):]

In [417]:
r = 5
Ntrees = 175
criterion = 'gini'
thetas = np.linspace(0.0,np.pi/4,r)
coords = np.zeros((train_ind.shape[0],r*2))
for i,theta in enumerate(thetas):
    coords[:,2*i:2*i+2] = Rotate(XY[train_ind,:],theta)
rforest = Forest(n_estimators=Ntrees, criterion=criterion,n_jobs=4)
tstart = time.time();
rforest.fit(coords,target[train_ind])
print time.time()-tstart

11.3985869884


In [418]:
coords = np.zeros((test_ind.shape[0],2*r))
for i,theta in enumerate(thetas):
    coords[:,2*i:2*i+2] = Rotate(XY[test_ind,:],theta)
rforest.score(coords,target[test_ind])

0.68164983164983162

In [321]:
coords = np.zeros((test_ind.shape[0],2*r))
for i,theta in enumerate(thetas):
    coords[:,2*i:2*i+2] = Rotate(XY[test_ind,:],theta)
#np.random.shuffle(coords)
rforest.score(np.concatenate([coords,Ru[test_ind].reshape((coords.shape[0],1)),Rb[test_ind].reshape((coords.shape[0],1))],axis=1),target[test_ind])

ValueError: Number of features of the model must  match the input. Model n_features is 6 and  input n_features is 8 

In [219]:
rforest.feature_importances_

array([ 0.14438164,  0.13979233,  0.13888256,  0.14194109,  0.14092658,
        0.13988656,  0.13665635,  0.0175329 ])

In [215]:
train_labels.groupby('status_group').count()/(1.0*N)

,id
status_group,
functional,0.543081
functional needs repair,0.072677
non functional,0.384242


In [343]:
# CONTOUR
nx = 1200; ny = 1200
xx,yy = np.meshgrid(np.linspace(35.0, 37,   nx),
                    np.linspace(-7.0, -9,   ny))
cmap = plt.cm.RdYlGn
xy = np.c_[xx.ravel(), yy.ravel()]
coords = np.zeros((xy.shape[0],2*r))
for i in xrange(r):
    coords[:,2*i:2*i+2] = Rotate(xy,thetas[i])
Z = rforest.predict(coords)
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, np.linspace(0.0,2.0,4), cmap=cmap)
plt.gca().set_aspect('equal'); plt.show(); 

In [341]:
# IMAGE
nx = 1000; ny = 1000
xx,yy = np.meshgrid(np.linspace(35.0, 37,   nx),
                    np.linspace(-7.0, -9,   ny))
cmap = plt.cm.RdYlGn
xy = np.c_[xx.ravel(), yy.ravel()]
coords = np.zeros((xy.shape[0],2*r))
for i in xrange(r):
    coords[:,2*i:2*i+2] = Rotate(xy,thetas[i])
Z = rforest.predict(coords)
Z = Z.reshape(xx.shape)
plt.pcolormesh(xx, yy, Z,cmap=cmap)
plt.gca().set_aspect('equal'); plt.show(); 

In [344]:
status_groups = ['functional', 'functional needs repair', 'non functional']
for i,c in enumerate(['green','yellow','red']):
    x=train_data[train_labels['status_group']==status_groups[i]]['longitude']
    y=train_data[train_labels['status_group']==status_groups[i]]['latitude']
    plt.plot(x,y,'o',color=c,markersize=6)
plt.gca().set_aspect('equal')
plt.gca().set_xlim((29,41))

(29, 41)

0.66708754208754206

In [104]:
### FOREST CLASSIFIER
myforest = Forest(n_estimators=100,criterion='entropy')
myforest.fit(Dgps[train_ind,:],target[train_ind])

RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='entropy', max_depth=None, max_features='auto',
            min_density=None, min_samples_leaf=1, min_samples_split=2,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0)

In [105]:
predictions = myforest.predict(Dgps[test_ind])
np.sum(predictions == target[test_ind])/(1.0*test_ind.shape[0])

0.7263468013468013

In [100]:
myforest.score(Dall[test_ind,:],target[test_ind])
myforest.feature_importances_

array([ 0.39320319,  0.05531299,  0.02785572,  0.014544  ,  0.10050808,
        0.01048493,  0.00414511,  0.04985363,  0.03122495,  0.0284956 ,
        0.00797111,  0.08312579,  0.13425121,  0.05902369,  0.        ,  0.        ])

In [268]:
R

array([ 0.79620773,  0.508857  ,  0.49224358, ...,  0.30268173,
        0.770254  ,  0.87442789])

In [107]:
plt.gcf().tight_layout()

In [106]:
plt.figure(); plt.gca().set_aspect("equal")
nx = 1000; ny = 1000
xx,yy = np.meshgrid(np.linspace(35.0, 37,   nx),
                    np.linspace(-7.0, -9,   ny))
probs = myforest.predict_proba(np.array([xx.ravel(),yy.ravel()]).T)[:,0]
probs = np.reshape(probs,(nx,ny))
plt.contourf(xx,yy,probs,np.linspace(0.0,1.0,5))
plt.plot(X[(train_labels.status_group!='non functional')],Y[(train_labels.status_group!='non functional')],'go');
plt.plot(X[(train_labels.status_group=='non functional')],Y[(train_labels.status_group=='non functional')],'ro'); plt.gca().set_aspect("equal")

In [121]:
probs

array([[ 0.25,  0.75],
       [ 0.26,  0.74],
       [ 0.23,  0.77],
       ..., 
       [ 0.35,  0.65],
       [ 0.5 ,  0.5 ],
       [ 0.24,  0.76]])

In [112]:
xx.ravel()

array([ 30.4       ,  30.40606061,  30.41212121, ...,  30.98787879,
        30.99393939,  31.        ])

In [86]:
myforest.feature_importances_

array([ 0.03387232,  0.00753619,  0.09897791,  0.00718113,  0.85243245])

In [62]:
### DECISION TREE CLASSIFIER
mytree = Tree()
mytree.fit(Dall[train_ind,:],train_labels.status_group[train_ind])

DecisionTreeClassifier(compute_importances=None, criterion='gini',
            max_depth=None, max_features=None, min_density=None,
            min_samples_leaf=1, min_samples_split=2, random_state=None,
            splitter='best')

In [63]:
predictions = mytree.predict(Dall[test_ind,:])
np.sum(predictions == train_labels.status_group[test_ind])/(1.0*N)

0.13488215488215488

In [57]:
mytree.feature_importances_

array([ 0.3226293 ,  0.32493605,  0.01043847,  0.00603807,  0.13204862,
        0.00486731,  0.19904219])

In [107]:
plt.plot(X[(train_labels.status_group!='non functional')],Y[(train_labels.status_group!='non functional')],'go');
plt.plot(X[(train_labels.status_group=='non functional')],Y[(train_labels.status_group=='non functional')],'ro'); plt.gca().set_aspect("equal")

In [346]:
######################### OVERLAY EACH INDIVIDUAL TREE FROM THE FOREST
nx = 100; ny = 100
xx,yy = np.meshgrid(np.linspace(35.0, 37,   nx),
                    np.linspace(-7.0, -9,   ny))
cmap = plt.cm.RdYlGn
model = rforest
estimator_alpha = 1.0 / len(model.estimators_)
coords = np.zeros((xy.shape[0],2*r))
for i in xrange(r):
    coords[:,2*i:2*i+2] = Rotate(xy,thetas[i])
for tree in model.estimators_:
    xy = np.c_[xx.ravel(), yy.ravel()]
    Z = tree.predict(coords)
    Z = Z.reshape(xx.shape)
    cs = plt.pcolormesh(xx, yy, Z, alpha=estimator_alpha, cmap=cmap)
plt.show()

In [389]:
status_groups = ['functional', 'functional needs repair', 'non functional']
for i,c in enumerate(['green','yellow','red']):
    x=train_data[train_labels['status_group']==status_groups[i]]['longitude']
    y=train_data[train_labels['status_group']==status_groups[i]]['latitude']
    plt.plot(x,y,'o',color=c,markersize=6)
plt.gca().set_aspect('equal')
plt.gca().set_xlim((29,41))

(29, 41)

In [384]:
r = 100
forests = []
ptot = np.zeros((test_ind.shape[0],2))
for theta in np.linspace(0,np.pi/4,r):
    f = Forest(n_estimators = 100/r,criterion='gini')
    f.fit(Rotate(XY[train_ind,:],theta),target[train_ind])
    forests += [f]

In [391]:
######################### OVERLAY EACH INDIVIDUAL TREE FROM THE FOREST
plt.figure()
nx = 500; ny = 500
xx,yy = np.meshgrid(np.linspace(35.2, 36.2,   nx),
                    np.linspace(-5, -4.00,    ny))
cmap = plt.cm.RdYlGn
model = rforest
estimator_alpha = 1.0 / len(model.estimators_)
for theta,forest in zip(np.linspace(0,np.pi/4,r),forests):
    xy = np.c_[xx.ravel(), yy.ravel()]
    Z = forest.predict(Rotate(xy,theta))
    Z = Z.reshape(xx.shape)
    cs = plt.contourf(xx, yy, Z, alpha=estimator_alpha, cmap=cmap)
plt.show()

In [359]:
target[0:20]

array([2, 2, 2, 0, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 1, 2])

In [385]:
p = np.zeros((test_ind.shape[0],3))
for i,theta in enumerate(np.linspace(0,np.pi/4,r)):
    p += forests[i].predict_proba(Rotate(XY[test_ind,:],theta))
prediction = np.argmax(p,axis=1)
np.sum(prediction == target[test_ind])/(1.0*test_ind.shape[0])

0.6835016835016835

array([2, 0, 2, ..., 2, 0, 2])